In [231]:
import pandas as pd                    
import xml.etree.ElementTree as ET
import unicodedata

N_CVs = 53
linhas = [] # uma linha para cada artigo lido
base_path = '../xml/'

researcher_df = pd.DataFrame(author_lst, columns=['name', 'research_area', 'program'])
base_research_field = pd.DataFrame(columns=['name', 'research_area'])

In [232]:
def normalize_str(source, encoding):
    if encoding  == 'ISO-8859-1':
        target = str.encode(source, 'ISO-8859-1')
        target = target.decode('UTF-8')
    else:
        target = str(source)
        
    target = target.strip()
    
    target = ''.join(c for c in unicodedata.normalize('NFD', target)
                  if unicodedata.category(c) != 'Mn')
    
    return target.upper()

In [233]:
def parse_researcher_data(xmlelement, field_df, target_df):
    name = normalize_str(xmlelement.get('NOME-COMPLETO'), 'ISO-8859-1')
    program = "PPGC" # Can change later to add a properly method to fetch
    field = "undef"
    print("RESEARCHER [" + name + "]\t[" + program + "]\t[" + field + "]")

In [237]:
with open(base_path + "Linhas_de_Pesquisa.csv", "r") as file:
    next(file)
    iter = 0
    for line in file:.first
        arr = line.split(';')
        base_research_field.loc[iter] = [normalize_str(arr[2], 'UTF-8'), arr[1]]
        iter += 1

In [243]:
base_research_field[base_research_field['name'] == 'ANDERSON MACIEL']

,name,research_area
0,ANDERSON MACIEL,Computação Gráfica e Visualização de Dados
11,ANDERSON MACIEL,"Interação Humano-Computador, Realidade Virtual..."


In [236]:
# Carrega CVs e cria um dataframe com os dados dos artigos lidos
for i in range(1,N_CVs+1):
    arquivo = base_path + 'curriculo(' + str(i) + ').xml'
    print("Lendo arquivo: ", arquivo)
    doc_tree_root = ET.parse(arquivo).getroot()
    for elem in doc_tree_root:                
        if elem.tag == 'PRODUCAO-BIBLIOGRAFICA':
            parse_production(linhas, elem)
        if elem.tag == 'DADOS-GERAIS':
            researcher_df = parse_researcher_data(elem, researcher_df)
prodPPGC = pd.DataFrame(linhas) # df é o dataframe contendo todos os dados lidos

Lendo arquivo:  ../xml/curriculo(1).xml
RESEARCHER [RENATO PEREZ RIBAS]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(2).xml
RESEARCHER [VALTER ROESLER]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(3).xml
RESEARCHER [LUIS DA CUNHA LAMB]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(4).xml
RESEARCHER [INGRID OLIVEIRA DE NUNES]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(5).xml
RESEARCHER [ERIKA FERNANDES COTA]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(6).xml
RESEARCHER [CLAUDIO ROSITO JUNG]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(7).xml
RESEARCHER [KARIN BECKER]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(8).xml
RESEARCHER [DALTRO JOSE NUNES]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(9).xml
RESEARCHER [JACOB SCHARCANSKI]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(10).xml
RESEARCHER [NICOLAS MAILLARD]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(11).xml
RESEARCHER [MARCELO SOARES PIMENTA]	[PPGC]	[undef]
Lendo arquivo:  ../xml/curriculo(12).xml
R

In [69]:
institution = pd.DataFrame(columns=['institution_name'])
production = pd.DataFrame(columns=['event_name', 'event_qualis', 'event_type'])
time = pd.DataFrame(columns=['year'])
fact_Table = pd.DataFrame(columns=['time_id', 'researcher_id', 'event_id', 'institution_id', 'count'])

In [107]:
author_lst = list()
for auth_lst in prodPPGC['AUTORES']:
    for auth in auth_lst:
        author_lst.append([normalize_str(auth, debug=True), '', 'PPGC'])

[Nabil Bennouri] becomes [NABIL BENNOURI]
[Jean Michel Karam] becomes [JEAN MICHEL KARAM]
[Bernard Courtois] becomes [BERNARD COURTOIS]
[Renato Perez Ribas] becomes [RENATO PEREZ RIBAS]
[Alvaro Bernal] becomes [ALVARO BERNAL]
[Alain Guyot] becomes [ALAIN GUYOT]
[Renato Perez Ribas] becomes [RENATO PEREZ RIBAS]
[Jerome Lescot] becomes [JEROME LESCOT]
[Jean Louis Leclercq] becomes [JEAN LOUIS LECLERCQ]
[Jean Michel Karam] becomes [JEAN MICHEL KARAM]
[Fabien Ndagijimana] becomes [FABIEN NDAGIJIMANA]
[Renato Perez Ribas] becomes [RENATO PEREZ RIBAS]
[Jerome Lescot] becomes [JEROME LESCOT]
[Fabien Ndagijimana] becomes [FABIEN NDAGIJIMANA]
[Jean Michel Karam] becomes [JEAN MICHEL KARAM]
[Renato Perez Ribas] becomes [RENATO PEREZ RIBAS]
[Damien Veychard] becomes [DAMIEN VEYCHARD]
[Jean Michel Karam] becomes [JEAN MICHEL KARAM]
[Bernard Courtois] becomes [BERNARD COURTOIS]
[Renato Perez Ribas] becomes [RENATO PEREZ RIBAS]
[Alvaro Bernal] becomes [ALVARO BERNAL]
[Alain Guyot] becomes [ALAIN GUY

438

In [62]:

researcher.drop_duplicates(subset='name', keep='last')

,name,research_area,program
0,Nabil Bennouri,,PPGC
13,Fabien Ndagijimana,,PPGC
17,Jean Michel Karam,,PPGC
33,Alvaro Bernal,,PPGC
36,Vasko Tchoumatchenko,,PPGC
37,Tania Vassileva,,PPGC
38,Alain Guyot,,PPGC
43,Luiz Antonio Razera Junior,,PPGC
54,Daniel Fachin,,PPGC
55,Ricardo Augusto da Luz Reis,,PPGC


In [207]:
def parse_production(linhas, elem):
    trabalhos = elem.findall('./TRABALHOS-EM-EVENTOS/TRABALHO-EM-EVENTOS')
    for trabalho in trabalhos:
        dados_basicos = trabalho.find('DADOS-BASICOS-DO-TRABALHO').attrib
        detalhes = trabalho.find('DETALHAMENTO-DO-TRABALHO').attrib
        autores = trabalho.findall('AUTORES')
        if dados_basicos.get('NATUREZA') == 'COMPLETO':
            #print('\tArtigo encontrado: ' + dados_basicos.get('TITULO-DO-TRABALHO'))                    
            dados = dict(dados_basicos)
            dados.update(detalhes)
            lst = []
            lst_autores= {}
            for autor in autores:                        
                lst.append(autor.get('NOME-COMPLETO-DO-AUTOR'))                      
            lst_autores['AUTORES'] = lst
            dados.update(lst_autores)                    
            linhas.append(dados)     